# 🎧 Beginner Audio Mixing in Python (Google Colab)
Welcome! This hands-on tutorial shows you how to:
1. Install lightweight audio libraries
2. Upload a backing track (a song)
3. Record your microphone **in Colab** (right from your browser)
4. Mix your voice with the song using **pydub**
5. Play and download the final mix

> **Tip:** Run each cell in order (press ▶️ on the left of each cell).


## What you'll learn
- The role of each lightweight library:
  - **pydub** — simple audio editing/mixing (overlay, gain, fade, export)
  - **soundfile** — read/write audio data
  - **IPython.display.Audio** — play audio in the notebook
- How to **upload** a song file
- How to **record** your mic from the browser into Colab
- How to **normalize levels** and **mix** tracks together
- How to **export** (save) your result as WAV/MP3


## 1) Setup: Install dependencies
We’ll install `pydub` and `soundfile`. We also install `ffmpeg`, which `pydub` uses behind the scenes to open and convert compressed formats (like MP3/WEBM/M4A).

In [ ]:
#@title Install libraries (run this once)
!apt-get -y install ffmpeg > /dev/null
!pip -q install pydub soundfile numpy > /dev/null

import os, sys, io, base64, numpy as np
from IPython.display import Audio, display, Javascript
from google.colab import files
from google.colab.output import eval_js
from pydub import AudioSegment
import soundfile as sf

print("✅ Setup complete. Libraries imported and ffmpeg installed.")

## 2) Upload your song (backing track)
Choose a music file from your computer (e.g. `.mp3`, `.wav`, `.m4a`). After upload, we store the **filename** so we can load it later with `pydub`.


In [ ]:
#@title Upload a song file
uploaded = files.upload()  # Use the file picker UI
if uploaded:
    song_filename = list(uploaded.keys())[0]
    print(f"Uploaded: {song_filename}")
else:
    song_filename = None
    print("No file uploaded yet.")

## 3) Record your microphone in the browser
This uses your browser’s mic to capture audio as **WEBM**, then we convert it to WAV internally.

- Press **Record** to start (you’ll see a permission prompt).
- Recording will automatically stop after the chosen duration.
- You can re-run this cell to record again.


In [ ]:
#@title Record microphone
duration_seconds = 8  #@param {type:"slider", min:3, max:60, step:1}
output_wav = "mic_recording.wav"  #@param {type:"string"}

# JavaScript function to capture microphone as WEBM and return base64
js = Javascript(r'''
async function recordAudio(sec=5) {
  const stream = await navigator.mediaDevices.getUserMedia({audio: true});
  const mediaRecorder = new MediaRecorder(stream);
  let chunks = [];
  mediaRecorder.ondataavailable = (e) => chunks.push(e.data);
  mediaRecorder.start();
  await new Promise(resolve => setTimeout(resolve, sec * 1000));
  mediaRecorder.stop();
  await new Promise(resolve => mediaRecorder.onstop = resolve);
  const blob = new Blob(chunks, {type: 'audio/webm'});
  const arrayBuffer = await blob.arrayBuffer();
  const base64String = btoa(String.fromCharCode(...new Uint8Array(arrayBuffer)));
  return base64String;
}
''')

display(js)

print("🎙️ Recording... Please wait", duration_seconds, "seconds")
base64_audio = eval_js(f"recordAudio({duration_seconds})")
print("✅ Recording finished. Converting to WAV...")

# Decode WEBM bytes
webm_bytes = base64.b64decode(base64_audio)

# Use pydub (ffmpeg) to read WEBM from memory, then export to WAV
audio = AudioSegment.from_file(io.BytesIO(webm_bytes), format="webm")
audio = audio.set_channels(1)  # mono mic is fine
audio = audio.set_frame_rate(44100)
audio.export(output_wav, format="wav")
print(f"Saved microphone recording to: {output_wav}")

# Quick playback
display(Audio(output_wav))

## 4) Load audio and basic checks
We’ll load both the **song** and **mic recording** with `pydub`. We’ll also resample them to common settings to avoid playback surprises.


In [ ]:
#@title Load files with pydub and normalize format
assert song_filename is not None, "Please upload a song in step 2."

# Load the song (any format supported by ffmpeg), and the mic (WAV)
song = AudioSegment.from_file(song_filename)
mic = AudioSegment.from_file("mic_recording.wav")

# Convert both to 44.1kHz stereo for consistency
song = song.set_frame_rate(44100).set_channels(2)
mic = mic.set_frame_rate(44100).set_channels(2)

print("Song duration (s):", round(len(song)/1000, 2))
print("Mic duration (s):", round(len(mic)/1000, 2))

## 5) Align lengths (optional)
Often your recording is shorter than the song. You can either:
- Trim the song to the mic length, or
- Loop/extend your mic (less common), or
- Keep the song full-length and only sing over a part.

Below we **trim the song** to the mic’s length for a quick demo. You can change this behavior.


In [ ]:
#@title Trim the song to the length of your mic (optional)
trim_to_mic = True  #@param {type:"boolean"}

if trim_to_mic:
    target_len = len(mic)  # milliseconds
    song_trimmed = song[:target_len]
else:
    song_trimmed = song

print("Working song length (s):", round(len(song_trimmed)/1000, 2))

## 6) Gain staging (set levels)
Basic mixing step: adjust volumes so neither track overwhelms the other.  
Use the sliders below to tweak:
- **song_gain_db**: negative values make the song quieter
- **mic_gain_db**: boost or reduce your vocal

> Start by turning the song down a bit (e.g. `-6 dB`) so your vocal is clearly heard.


In [ ]:
#@title Adjust gain levels
song_gain_db = -6.0  #@param {type:"slider", min:-24, max:12, step:0.5}
mic_gain_db = 3.0    #@param {type:"slider", min:-24, max:12, step:0.5}

song_ready = song_trimmed + song_gain_db
mic_ready = mic + mic_gain_db

print(f"Applied {song_gain_db} dB to song, {mic_gain_db} dB to mic.")

## 7) Optional: fades
Fades help avoid clicks when audio starts/stops abruptly.

In [ ]:
#@title Apply fades (optional)
apply_fades = True  #@param {type:"boolean"}
fade_in_ms = 1000   #@param {type:"slider", min:0, max:5000, step:100}
fade_out_ms = 1000  #@param {type:"slider", min:0, max:5000, step:100}

if apply_fades:
    song_ready = song_ready.fade_in(fade_in_ms).fade_out(fade_out_ms)
    mic_ready = mic_ready.fade_in(fade_in_ms).fade_out(fade_out_ms)

print("Fades applied." if apply_fades else "No fades applied.")

## 8) Mix (overlay) the tracks
We’ll **overlay** the vocal on top of the song. If your vocal starts a bit late, you can add a delay offset.


In [ ]:
#@title Create the mix
vocal_delay_ms = 0  #@param {type:"slider", min:0, max:5000, step:100}

# Optionally pad the vocal with silence if you want it to start later
if vocal_delay_ms > 0:
    mic_ready = AudioSegment.silent(duration=vocal_delay_ms) + mic_ready

# Make sure both have same length for simple export
mix_len = min(len(song_ready), len(mic_ready))
mix = song_ready[:mix_len].overlay(mic_ready[:mix_len])

print("✅ Mix created.")

## 9) Export and listen
We’ll export to **WAV** (lossless) and also show how to export to **MP3**.
Then we’ll play the WAV inside Colab.


In [ ]:
#@title Export files and play result
final_wav = "final_mix.wav"
final_mp3 = "final_mix.mp3"

mix.export(final_wav, format="wav")
print(f"Saved: {final_wav}")

# MP3 needs bitrate; pydub uses ffmpeg to encode
mix.export(final_mp3, format="mp3", bitrate="192k")
print(f"Saved: {final_mp3}")

# In-notebook player
display(Audio(final_wav))

## 10) Download your files
Click to download the **final WAV/MP3** or your original **mic recording**.


In [ ]:
#@title Download helpers
from google.colab import files

print("Use these buttons to download your files:")
# These will open file pickers to save locally
files.download("final_mix.wav")
files.download("final_mix.mp3")
files.download("mic_recording.wav")

## Extras & Troubleshooting

- **My mic is too quiet/loud** → Adjust `mic_gain_db` and `song_gain_db` in step 6.
- **Song and voice out of sync** → Try increasing `vocal_delay_ms` in step 8.
- **Clicks at start/end** → Enable fades in step 7.
- **Different file types** → `pydub` + `ffmpeg` handle many formats (MP3, WAV, M4A, OGG, WEBM).
- **Beginner tip** → Keep your peaks around **-6 dB to -3 dB** for clarity and headroom.

### How it works (short version)
- `pydub.AudioSegment` reads audio and treats it like a “waveform” you can add, slice, and overlay.
- `.overlay()` mixes two audio segments together (like stacking tracks in a DAW).
- Gain in **dB** is logarithmic; `+6 dB` ≈ double amplitude, `-6 dB` ≈ half.
- `ffmpeg` lets `pydub` handle compressed files and format conversions behind the scenes.

Happy mixing! 🎶
